In [1]:
import endomill
from nbmetalog import nbmetalog as nbm


In [2]:
nbm.print_metadata()


context: ci
hostname: 6bbf4a479cf4
interpreter: 3.8.12 (default, Jan 15 2022, 18:39:47)  [GCC 7.5.0]
nbcellexec: 2
nbname: phylogeny_reconstruction_mill
nbpath: /opt/hereditary-stratigraph-concept/binder/phylogenetic-inference/phylogeny_reconstruction_mill.ipynb
revision: null
session: 88c1a650-c203-43df-8ab4-45554e4b8e2f
timestamp: 2022-05-21T05:58:18Z00:00


IPython==7.16.1
keyname==0.4.1
yaml==5.3.1
endomill==0.1.3
nbmetalog==0.2.6
re==2.2.1
ipython_genutils==0.2.0
logging==0.5.1.2
zmq==22.3.0
json==2.0.9
ipykernel==5.5.3


In [3]:
base_url = (
    'https://github.com/mmore500/hereditary-stratigraph-concept/'
    'blob/f21c71cc31c6cc3eed4bbd47bfce2208020ce930/'
    'binder/phylogenetic-inference/'
)

data_urls = [
    base_url + 'a=pairwise_mrca_estimates+source=nk_ecoeaselection_seed110_pop100_mut.01_snapshot_3000.csv.gz',
    base_url + 'a=pairwise_mrca_estimates+source=nk_lexicaseselection_seed110_pop165_mut.01_snapshot_500.csv.gz',
    base_url + 'a=pairwise_mrca_estimates+source=nk_randomselection_seed7_pop100_mut.01_snapshot_5000.csv.gz',
    base_url + 'a=pairwise_mrca_estimates+source=nk_sharingselection_seed10_pop100_mut.01_snapshot_5000.csv.gz',
    base_url + 'a=pairwise_mrca_estimates+source=nk_tournamentselection_seed140_pop100_mut.01_snapshot_5000.csv.gz',
]

endomill.instantiate_over(
    parameter_packs=[
        {'data_url': data_url}
        for data_url in data_urls
    ],
)


👷🪓 milling {'data_url': 'https://github.com/mmore500/hereditary-stratigraph-concept/blob/f21c71cc31c6cc3eed4bbd47bfce2208020ce930/binder/phylogenetic-inference/a=pairwise_mrca_estimates+source=nk_ecoeaselection_seed110_pop100_mut.01_snapshot_3000.csv.gz'} ...


<IPython.core.display.Javascript object>

Input notebook does not contain a cell with tag 'parameters'


🪵✨ milled a=phylogeny_reconstruction+source=nk_ecoeaselection_seed110_pop100_mut.01_snapshot_3000.csv.gz.endomill.ipynb



👷🪓 milling {'data_url': 'https://github.com/mmore500/hereditary-stratigraph-concept/blob/f21c71cc31c6cc3eed4bbd47bfce2208020ce930/binder/phylogenetic-inference/a=pairwise_mrca_estimates+source=nk_lexicaseselection_seed110_pop165_mut.01_snapshot_500.csv.gz'} ...


...

Input notebook does not contain a cell with tag 'parameters'


🪵✨ milled a=phylogeny_reconstruction+source=nk_tournamentselection_seed140_pop100_mut.01_snapshot_5000.csv.gz.endomill.ipynb



🏠🍻 milling complete!


<IPython.core.display.Javascript object>

🌛💤 disabling notebok by overriding IPython with nop runner


In [4]:
#define papermil parameters
data_url: str


In [5]:
import alifedata_phyloinformatics_convert as apc
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceMatrix, DistanceTreeConstructor
from collections import Counter
from iterpop import iterpop as ip
import itertools as it
from keyname import keyname as kn
from matplotlib import pyplot as plt
import numpy as np
from os.path import basename
import pandas as pd
import sys
from teeplot import teeplot as tp
from tqdm import tqdm
import warnings


In [6]:
from pylib import draw_biopython_tree_with_origin_time_bounds


In [7]:
nbm.print_metadata()


In [8]:
df = pd.read_csv(
    f'{data_url}?raw=true',
    compression='gzip',
)

nbm.print_dataframe_synopsis(df)

data_filename = basename(data_url)

print(data_filename)


In [9]:
endomill.add_instance_outpath(
    f'a=phylogeny_reconstruction+source={kn.unpack(data_filename)["source"]}.endomill.ipynb'
)


In [10]:
df['Treatment'] = df.apply(
    lambda row: kn.pack({
        'target': row['Stratigraphic Column Target Retained Bits'],
        'differentia': row['Differentia Bit Width'],
        'policy': row['Stratum Retention Policy'],
    }),
    axis=1,
)


In [11]:
max_gen = df['Generation of Taxon Compared From'].max()


In [12]:
def create_distance_matrix(df):
    assert len(df['Treatment'].unique() == 1)

    counter = it.count()
    taxon_to_index = {
        taxon : next(counter)
        for taxon in df['Taxon Compared From'].unique()
    }
    label_list = sorted(
        [*df['Taxon Compared From'].unique()],
        key=lambda x: taxon_to_index[x],
    )
    assert len(label_list) == len(taxon_to_index)

    def row_to_mrca_time_mid(row):
        mrca_lb = row['Generation Of MRCA Lower Bound (inclusive)']
        mrca_ub = row['Generation Of MRCA Upper Bound (exclusive)']
        earliest_detectable_mrca = row['Rank of Earliest Detectable Mrca With']
        taxon1_gen = row['Generation of Taxon Compared From']
        taxon2_gen = row['Generation of Taxon Compared To']

        if not np.isnan(mrca_lb) and not np.isnan(mrca_ub):
            return (mrca_lb + mrca_ub - 1) / 2 # ub is exclusive
        elif earliest_detectable_mrca == 0:
            # we are confident that no mrca exists, but return 0
            return 0
        elif np.isnan(mrca_ub) and not np.isnan(earliest_detectable_mrca):
            assert np.isnan(mrca_lb)
            return (earliest_detectable_mrca - 1) / 2 # exclusive
        else:
            assert np.isnan(mrca_lb) and np.isnan(mrca_ub) and np.isnan(earliest_detectable_mrca)
            ub = min(taxon1_gen, taxon2_gen)
            return ub / 2
    mrca_time_mid_dict = {
        (row['Taxon Compared From'], row['Taxon Compared To']):
             row_to_mrca_time_mid(row)
        for __, row in df.iterrows()
    }
    assert not any(np.isnan(x) for x in mrca_time_mid_dict.values())

    distance_dict = {
        (taxon_to_index[row['Taxon Compared From']], taxon_to_index[row['Taxon Compared To']]):
            row['Generation of Taxon Compared From']
            + row['Generation of Taxon Compared To']
            - 2 * mrca_time_mid_dict[
                (row['Taxon Compared From'], row['Taxon Compared To'])
            ]
        for __, row in df.iterrows()
    }
    assert not any(np.isnan(x) for x in distance_dict.values())

    distance_matrix = np.array([
        [
            float(distance_dict[(i, j)] + distance_dict[(j, i)])
            if i != j
            else 0.0
            for j in range(len(taxon_to_index))
        ]
        for i in range(len(taxon_to_index))
    ])

    return distance_matrix, label_list


In [13]:
def to_tril(matrix):
    return [
        row[:row_idx] + [0]
        for row_idx, row in enumerate(matrix.tolist())
    ]


In [14]:
def distance_construct_tree(df, algorithm):
    def row_to_mrca_time_lb(row):
        mrca_lb = row['Generation Of MRCA Lower Bound (inclusive)']
        if not np.isnan(mrca_lb):
            return mrca_lb
        else:
            assert np.isnan(row['Generation Of MRCA Upper Bound (exclusive)'])
            return 0
    mrca_time_lb_dict = {
        (row['Taxon Compared From'], row['Taxon Compared To']):
            row_to_mrca_time_lb(row)
        for __, row in df.iterrows()
    }
    assert not any(np.isnan(x) for x in mrca_time_lb_dict.values())

    def row_to_mrca_time_ub(row):
        mrca_ub = row['Generation Of MRCA Upper Bound (exclusive)']
        earliest_detectable_mrca = row['Rank of Earliest Detectable Mrca With']
        taxon1_gen = row['Generation of Taxon Compared From']
        taxon2_gen = row['Generation of Taxon Compared To']
        if not np.isnan(mrca_ub):
            return mrca_ub
        elif not np.isnan(earliest_detectable_mrca):
            return earliest_detectable_mrca
        else:
            assert np.isnan(mrca_ub) and np.isnan(earliest_detectable_mrca)
            return min(taxon1_gen, taxon2_gen) + 1 #exclusive
    mrca_time_ub_dict = {
        (row['Taxon Compared From'], row['Taxon Compared To']):
            row_to_mrca_time_ub(row)
        for __, row in df.iterrows()
    }
    assert not any(np.isnan(x) for x in mrca_time_ub_dict.values())

    for __, row in df.iterrows():
        mrca_time_lb_dict[(row['Taxon Compared From'], row['Taxon Compared From'])] \
            = row['Generation of Taxon Compared From'] # inclusive
        mrca_time_ub_dict[(row['Taxon Compared From'], row['Taxon Compared From'])] \
            = row['Generation of Taxon Compared From'] + 1 # exclusive


    assert len(df['Treatment'].unique() == 1)
    distance_matrix, label_list = create_distance_matrix(df)
    assert not np.isnan(distance_matrix.sum())
    dm = DistanceMatrix([*map(str, label_list)], to_tril(distance_matrix))
    tree = getattr(DistanceTreeConstructor(), algorithm)(dm)

    # fixup orign times
    for node in tree.find_clades():
        node.origin_time_lbs = []
        node.origin_time_ubs = []

    for leaf1, leaf2 in it.product(tree.get_terminals(), tree.get_terminals()):
        mrca_node = tree.common_ancestor(leaf1, leaf2)
        if leaf1 == leaf2:
            assert mrca_node == leaf1

        mrca_time_lb = mrca_time_lb_dict[(int(leaf1.name), int(leaf2.name))]
        assert not np.isnan(mrca_time_lb)
        mrca_node.origin_time_lbs.append(mrca_time_lb)

        mrca_time_ub = mrca_time_ub_dict[(int(leaf1.name), int(leaf2.name))]
        assert not np.isnan(mrca_time_ub)
        mrca_node.origin_time_ubs.append(mrca_time_ub)

    for node in tree.find_clades():
        assert node.origin_time_lbs
        assert node.origin_time_ubs

        # upper bound is a hard bound,
        # so anything above the lowest upper bound
        # is strictly impossible
        node.origin_time_ub = np.min(node.origin_time_ubs)

        # to calculate the consolidated lower bound
        # find all bounds with an upper component that is
        # has not been undercut
        # and then pick the strictest among those
        node.origin_time_lb = np.max([
            lb
            for lb, ub in zip(
                node.origin_time_lbs,
                node.origin_time_ubs,
            )
            if ub <= node.origin_time_ub
        ])

        assert node.origin_time_ub > node.origin_time_lb

        node.origin_time_est = np.mean([node.origin_time_lb, node.origin_time_ub - 1])

    # reroot tree
    new_root = min(tree.find_clades(), key=lambda node: node.origin_time_est)
    tree.root_with_outgroup(new_root)

    # fixup branch lengths
    tree.root.branch_length = tree.root.origin_time_est
    for node in tree.find_clades():
        for child in node.clades:
            child.branch_length = child.origin_time_est - node.origin_time_est

    # sort clades for consistent display
    for node in tree.find_clades(order='postorder'):
        if node.clades:
            node.max_descendant = max(child.max_descendant for child in node.clades)
        else:
            node.max_descendant = int(node.name)
        node.clades.sort(key=lambda node: node.max_descendant)

    return tree


In [15]:
def nj(df): return distance_construct_tree(df, 'nj')
def upgma(df): return distance_construct_tree(df, 'upgma')

trees = {
    (treatment, algorithm): algorithm(group)
    for treatment, group in tqdm(df.groupby('Treatment'))
    for algorithm in (
        # disabled nj due to obvious discrepancies
        # with respect to origin time of internal nodes
        # that break assertions
        # nj,
        upgma,
    )
}


In [16]:
taxon_rename_dict = {
    str(taxon): np.base_repr(i, 36).lower()
    for i, taxon
    in enumerate(sorted(df['Taxon Compared From'].unique()))
}
for (treatment, algorithm), tree in trees.items():
    tp.tee(
        draw_biopython_tree_with_origin_time_bounds,
        tree,
        label_func=lambda node: None if 'Inner' in str(node) else taxon_rename_dict[str(node)],
        teeplot_outattrs={
            'algorithm' : algorithm.__name__,
            'treatment' : kn.demote(treatment),
        },
    )
    plt.show()


In [17]:
alife_dataframes = {
    (treatment, algorithm): apc.biopython_tree_to_alife_dataframe(
        tree,
        exportattrs={
            'origin_time_lb': 'Origin Time Lower Bound (Inclusive)',
            'origin_time_ub': 'Origin Time Upper Bound (Exclusive)',
            'origin_time_est': 'Origin Time Estimate',
        }
    )
    for (treatment, algorithm), tree in trees.items()
}
for (treatment, algorithm), df in alife_dataframes.items():
    df['Treatment'] = treatment
    df['Algorithm'] = algorithm


In [18]:
res_df = pd.concat(alife_dataframes.values()).reset_index()


In [19]:
res_df


In [20]:
nbm.print_dataframe_synopsis(res_df)


In [21]:
res_df.to_csv(
    f'a=reconstructed_phylogenies+source={kn.unpack(data_filename)["source"]}',
    compression='gzip',
)
res_df.to_csv(
    f'a=reconstructed_phylogenies+source={kn.unpack(data_filename)["source"][:-len(".gz")]}',
)
